In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Encoder + GBC
The idea of using an encoder is the denoise the data. We use the encoded features in a gradient boosting classifier. 

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
import random

import kerastuner as kt

from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

## PurgedGroupTimeSeriesSplit
Click the code button to see. 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

### Loading the training data

In [ ]:
import gc

TRAINING = False
FOLDS = 7
SEED = 42

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train = train.query('weight > 0').reset_index(drop = True)

features = [c for c in train.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
EPSILON = {c:0.0 for c in resp_cols}

In [ ]:
X = train[features].values
y = np.stack([(train[c] > EPSILON[c]).astype('int') for c in resp_cols]).T 
y = np.around(np.mean(y,axis=-1)).reshape((-1,1))

f_mean = np.mean(train[features[1:]].values,axis=0)
dates = train['date'].values

In [ ]:
del train
gc.collect()

### Creating the autoencoder. 
The autoencoder should aid in denoising the data. Based on [this](https://www.semanticscholar.org/paper/Deep-Bottleneck-Classifiers-in-Supervised-Dimension-Parviainen/fb86483f7573f6430fe4597432b0cd3e34b16e43) paper. 

In [ ]:
u = np.unique(y,return_counts=True)[1]
th = u[1]/sum(u)
th

In [ ]:
def create_encoder(hp,input_dim,output_dim):
    i = Input(input_dim)
    x = BatchNormalization()(i)
    x = GaussianNoise(hp.Float('noise',0.1,0.5,default=0.1))(x)
    
    encoding_units = []
    encoding_dp = []
    for j in range(hp.Int('num_encoding_layers',1,3)):
        u = hp.Int(f'encoding_units_{j}',64,512)
        encoding_units.append(u)
        x = Dense(u,activation='relu')(x)
        x = BatchNormalization()(x)
        dp = hp.Float(f'encoding_dp_{j}',0.1,0.5)
        encoding_dp.append(dp)
        x = Dropout(dp)(x)
    
    encoder = Model(inputs=i,outputs=x)
    
    for u,dp in zip(encoding_units[::-1],encoding_dp[::-1]):
        x = Dense(u,activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(dp)(x)
        
    decoded = Dense(input_dim,activation='linear',name='decoded')(x)
    
    for j in range(hp.Int('num_bottleneck_layers',1,3)):
        x = Dense(hp.Int(f'bottleneck_units_{j}',16,64))(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(hp.Float(f'bottleneck_dp_{j}',0.1,0.5))(x)
    
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.01),loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, encoder

In [ ]:
%%time
SEEDS = [123]

hp = kt.HyperParameters()

gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20,
                                         max_train_group_size=100,
                                         max_test_group_size=10)
splits = list(gkf.split(y, groups=dates))

fold_weights = np.asarray(list(map(lambda x: len(x[0]),splits)))
fold_weights = fold_weights/fold_weights.sum()
print(fold_weights.sum())


if TRAINING:
    for j,SEED in enumerate(SEEDS):
        set_all_seeds(SEED)
        
        oof = np.zeros(y.shape)
        for fold, (train_indices, test_indices) in enumerate(splits):
            X_train, X_test = X[train_indices], X[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            
            autoencoder,encoder = create_encoder(hp,X.shape[-1],y.shape[-1])
            autoencoder.fit(X_train,(X_train,y_train),validation_data=(X_test,(X_test,y_test)),epochs=100,batch_size=4096,verbose=2,
                      callbacks=[EarlyStopping('val_loss',patience=10,restore_best_weights=True),
                                 ReduceLROnPlateau('val_loss',patience=5)])
            
            encoder.save_weights(f'./encoder_{fold}.tf')
            
            X_train = encoder.predict(X_train)
            X_test = encoder.predict(X_test)
            
            tf.keras.backend.clear_session()
            
            clf = GradientBoostingClassifier(n_estimators=100,random_state=SEED,verbose=1)
            clf.fit(X_train, y_train.ravel())
            
            pd.to_pickle(clf,f'./gb_{fold}.pkl')

else:
    models = []
    encoders = []
    for SEED in SEEDS:
        for fold in range(FOLDS):
            _,encoder = create_encoder(hp,X.shape[-1],y.shape[-1])
            encoder.load_weights(f'../input/jsautoencodergb/encoder_{fold}.tf')
            encoder.call = tf.function(encoder.call,experimental_relax_shapes=True)
            encoders.append(encoder)
            model = pd.read_pickle(f'../input/jsautoencodergb/gb_{fold}.pkl')
            models.append(model)

## Submission

In [ ]:
predictions = []

In [ ]:
if not TRAINING:
    import janestreet
    janestreet.competition.make_env.__called__ = False
    env = janestreet.make_env()
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
                
            pred = np.sum(fold_weights * np.asarray([models[i].predict_proba(encoders[i](x_tt,training=False).numpy()).ravel()[1] for i in range(len(models))]))
            pred = np.where(pred > th, 1, 0).astype(int)
            predictions.append(pred)
            pred_df.action = pred
        else:
            pred_df.action = 0
        env.predict(pred_df)

In [ ]:
np.unique(predictions,return_counts=True)